<a href="https://colab.research.google.com/github/ItsShi/Mono-Depth/blob/main/dpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /content/MonoDepth
!git clone https://github.com/ItsShi/MonoDepth.git

!pip -q install tensorboardX==1.4
!pip -q install transformers
!pip -q install torchvision==0.12.0

Cloning into 'MonoDepth'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 224 (delta 73), reused 106 (delta 56), pack-reused 84
Receiving objects: 100% (224/224), 10.51 MiB | 32.42 MiB/s, done.
Resolving deltas: 100% (100/100), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.8 MB/s eta 0:00:00
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.1 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [3]:
import gdown
d3k1_url = 'https://drive.google.com/uc?id=17k1-CHptG_XTXVX8qp7yK_zadH54vVnY'
gdown.download(d3k1_url,'d3k1_rgb.mp4',quiet=True) 

'd3k1_rgb.mp4'

In [4]:
# vid2frame
!pip install ffmpeg
!mkdir /content/data
!ffmpeg -i /content/d3k1_rgb.mp4 %6d.jpg  
!mv *.jpg /content/data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=3891bb3c2c3ea17d9dcaa13c3d965e885cfc40b4fbe15f9e5e0eb1fd99b0a265
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontc

In [13]:
# depth gt
!pwd
import gdown
import shutil

scene_url = 'https://drive.google.com/uc?id=1xZwB0qFWbU7EjjI1g-Y0pqAs1_nWwU6Z'
gdown.download(scene_url, 'scene.tar.gz', quiet=True)

shutil.unpack_archive('scene.tar.gz', 'd3k1_tiffs')

/content


In [5]:
!touch train_files.txt
!touch val_files.txt
!touch test_files.txt

for opt in ["train", "val", "test"]:
    f = open("/content/MonoDepth/splits/endovis/{}_files.txt".format(opt), "r+")
    # f = open("/content/{}_files_original.txt".format(opt), "r+")
    new_file = []
    for line in f:
      folder, frame = line.split()[0], line.split()[1]
      if folder == "dataset3/keyframe1" or folder == "dataset3/keyframe3" or folder == "dataset3/keyframe4" and int(frame)<328:
          new_file.append(line)
    print(len(new_file))
    with open("{}_files.txt".format(opt), "w+") as f:
      for i in new_file:
        f.write(i)

# move and replace the original train_files.txt file
%pwd
!cp /content/MonoDepth/splits/endovis/test_files.txt /content/test_files_original.txt
!cp /content/MonoDepth/splits/endovis/train_files.txt /content/train_files_original.txt
!cp /content/MonoDepth/splits/endovis/val_files.txt /content/val_files_original.txt
!cp -f /content/val_files.txt /content/MonoDepth/splits/endovis/val_files.txt
!cp -f /content/train_files.txt /content/MonoDepth/splits/endovis/train_files.txt
!cp -f /content/test_files.txt /content/MonoDepth/splits/endovis/test_files.txt

290
284
34


In [108]:
!touch train_files.txt
!touch val_files.txt
!touch test_files.txt

for opt in ["train"]:
    f = open("/content/MonoDepth/splits/endovis/{}_files.txt".format(opt), "r+")
    # f = open("/content/{}_files_original.txt".format(opt), "r+")
    new_file = []
    for line in f:
      folder, frame = line.split()[0], line.split()[1]
      if int(folder[7]) < 8:
          new_file.append(line)
    print(len(new_file))
    with open("{}_files.txt".format(opt), "w+") as f:
      for i in new_file:
        f.write(i)

9480


In [6]:
%cd MonoDepth
!python test_simple.py --image_path assets/test_image.jpg --model_name mono_640x192
%cd ..
!cp -r MonoDepth/models/mono_640x192 mono_640x192

/content/MonoDepth
2023-06-14 16:05:20.318090: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
-> Downloading pretrained model to models/mono_640x192.zip
   Unzipping model...
   Model unzipped to models/mono_640x192
-> Loading model from  models/mono_640x192
   Loading pretrained encoder
   Loading pretrained decoder
-> Predicting on 1 test images
   Processed 1 of 1 images - saved predictions to:
   - assets/test_image_disp.jpeg
   - assets/test_image_disp.npy
-> Done!
/content


In [100]:
%cd MonoDepth
!python train.py --split endovis --data_path /content/data --dataset endovis --num_epochs 1 --log_frequency 20 --log_dir models --width 320 --height 256 --batch_size 4 --num_workers 0 --load_weights_folder /content/mono_640x192
%cd ..

/content/MonoDepth
2023-06-12 04:40:24.328385: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
loading model from folder /content/mono_640x192
Loading encoder weights...
Loading depth weights...
Loading pose_encoder weights...
Loading pose weights...
Cannot find Adam weights so Adam is randomly initialized
Training pose model named:
   mdp
Models and tensorboard events files are saved to:
   models
Training is using:
   cuda
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
Using split:
   endovis
There are 290 training items and 284 validation items

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite orde

In [14]:
# train with cv weights, save weights to mdp
%cd MonoDepth
!python train.py --dpt --split endovis --data_path /content/data --dataset endovis  --num_epochs 1 --log_frequency 20 --log_dir models --model_name mdp --width 320 --height 256 --batch_size 4 --num_workers 0 --load_weights_folder /content/mono_640x192 --models_to_load "pose_encoder" "pose" --scales 0
%cd ..

/content/MonoDepth
2023-06-14 16:26:29.125132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/transformers/models/dpt/feature_extraction_dpt.py:28: FutureWarning: The class DPTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DPTImageProcessor instead.
  warnings.warn(
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
loading model from folder /content/mono_640x192
Loading pose_encoder weights...
Loading pose wei

In [16]:
# finetune on weights /content/MonoDepth/models/mdp/models/weights_0 
%cd MonoDepth
!python train.py --dpt --split endovis --data_path /content/data --dataset endovis  --num_epochs 1 --log_frequency 20 --log_dir models --model_name dpt --width 320 --height 256 --batch_size 4 --num_workers 0 --load_weights_folder /content/MonoDepth/models/mdp/models/weights_0 --models_to_load "depth" "pose_encoder" "pose" --scales 0
%cd ..

/content/MonoDepth
2023-06-14 16:39:56.141509: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/transformers/models/dpt/feature_extraction_dpt.py:28: FutureWarning: The class DPTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DPTImageProcessor instead.
  warnings.warn(
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution2.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
loading model from folder /content/MonoDepth/models/mdp/models/weights_0
Loading depth weights..

In [15]:
%cd MonoDepth
!python export_gt_depth.py --data_path /content/d3k1_tiffs --split endovis
%cd ..

/content/MonoDepth
Exporting ground truth depths for endovis
1
dataset3/keyframe4
2
dataset3/keyframe4
3
dataset3/keyframe4
4
dataset3/keyframe4
5
dataset3/keyframe4
6
dataset3/keyframe4
7
dataset3/keyframe4
8
dataset3/keyframe4
9
dataset3/keyframe4
10
dataset3/keyframe4
11
dataset3/keyframe4
12
dataset3/keyframe4
13
dataset3/keyframe4
14
dataset3/keyframe4
15
dataset3/keyframe4
16
dataset3/keyframe4
17
dataset3/keyframe4
18
dataset3/keyframe4
19
dataset3/keyframe4
20
dataset3/keyframe4
21
dataset3/keyframe4
22
dataset3/keyframe4
23
dataset3/keyframe4
24
dataset3/keyframe4
25
dataset3/keyframe4
26
dataset3/keyframe4
27
dataset3/keyframe4
28
dataset3/keyframe4
29
dataset3/keyframe4
30
dataset3/keyframe4
31
dataset3/keyframe4
32
dataset3/keyframe4
33
dataset3/keyframe4
34
dataset3/keyframe4
Saving to endovis
/content


In [37]:
%cd MonoDepth
!python evaluate_depth.py --eval_split endovis --data_path /content/data --eval_mono  --width 320 --height 256 --load_weights_folder /content/mono_640x192
%cd ..

/content/MonoDepth
2023-06-14 18:49:56.862303: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
-> Loading weights from /content/mono_640x192
Dataset size HxW:  192 640
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
-> Computing predictions with size 640x192
MIN MAX of depth:  tensor(1.0079, device='

In [36]:
# /content/MonoDepth/models/mdp/models/weights_0 
# 
%cd MonoDepth
!python evaluate_depth.py --dpt --eval_split endovis --data_path /content/data --eval_mono  --width 320 --height 256 --load_weights_folder /content/MonoDepth/models/mdp/models/weights_0 
%cd ..

/content/MonoDepth
2023-06-14 18:46:29.878097: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
-> Loading weights from /content/MonoDepth/models/mdp/models/weights_0
Dataset size HxW:  256 320
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/transformers/models/

In [1]:
!pip -q install transformers
!rm -rf /content/MonoDepth
!git clone https://github.com/ItsShi/MonoDepth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.1 MB/s eta 0:00:00
Cloning into 'MonoDepth'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 221 (delta 72), reused 106 (delta 56), pack-reused 84
Receiving objects: 100% (221/221), 10.51 MiB | 21.65 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [39]:
import gdown
url = 'https://drive.google.com/uc?id=1bgrGt-DM6xQ_eSkI8etMcV_SeyO2ZtMb'
gdown.download(url,'dpt_weights_7.zip',quiet=True) 
!unzip -q dpt_weights_7.zip

In [40]:
# /content/weights_7
%cd MonoDepth
!python evaluate_depth.py --dpt --eval_split endovis --data_path /content/data --eval_mono  --width 320 --height 256 --load_weights_folder /content/weights_7
%cd ..

/content/MonoDepth
2023-06-14 18:55:45.092447: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
-> Loading weights from /content/weights_7
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/transformers/models/dpt/feature_extraction_dpt.py:28: FutureWarning: The cl

In [27]:
import gdown

gt_url = 'https://drive.google.com/uc?id=1GHX4ML7kUP4xFZSi56VtpZSxZdwHa2oO'
gdown.download(gt_url,'gt.zip',quiet=True) 
!unzip -q gt.zip

In [7]:
# /content/MonoDepth/models/dpt/models/weights_0 
# /content/weights_7
%cd MonoDepth
!python test_simple.py --dpt --ext png --image_path /content/SCARED/d5k4_Left_Image.png --model_name /content/weights_7
%cd ..

/content/MonoDepth
2023-06-14 19:37:40.001232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
-> Loading model from  /content/weights_7
/usr/local/lib/python3.10/dist-packages/transformers/models/dpt/feature_extraction_dpt.py:28: FutureWarning: The class DPTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DPTImageProcessor instead.
  warnings.warn(
   Loading pretrained decoder
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
-> Predicting on 1 test 

In [9]:
%cd MonoDepth
!python test_simple.py --dpt --ext png --image_path /content/SCARED/d5k4_Left_Image.png 
%cd ..

/content/MonoDepth
2023-06-14 19:40:47.115297: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/MonoDepth/test_simple.py", line 185, in <module>
    test_simple(args)
  File "/content/MonoDepth/test_simple.py", line 53, in test_simple
    assert args.model_name is not None, \
AssertionError: You must specify the --model_name parameter; see README.md for an example
/content


In [1]:
%cd MonoDepth
!python test_simple.py --image_path assets/test_image.jpg --model_name mono_640x192
%cd ..

/content/MonoDepth
2023-06-14 19:27:40.757572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
-> Loading model from  models/mono_640x192
   Loading pretrained encoder
   Loading pretrained decoder
-> Predicting on 1 test images
torch.Size([1, 1, 192, 640]) 235 638
   Processed 1 of 1 images - saved predictions to:
   - assets/test_image_disp.jpeg
   - assets/test_image_disp.npy
-> Done!
/content
